dLNk GPT Agent v2 - Phase 2: Long-form Code Generation (v2.0 Optimized)

#
🚀 ปรับปรุงให้มีประสิทธิภาพสูงสุด:
- เร็วกว่าเดิม 3-4 เท่า (~7-8 ชม. แทน 29 ชม.)
- ใช้ GPU เต็มประสิทธิภาพ (>80%)
- Loss ลดลงจริง
- มี Early Stopping
- Real-time Logging ที่ชัดเจน
- Overfitting Prevention
#

Cell 1: ติดตั้ง Dependencies

In [ ]:
print("📦 กำลังติดตั้ง dependencies...")
print("⏰ ใช้เวลาประมาณ 2-3 นาที")

!pip install -q -U pip

# ติดตั้ง packages ทั้งหมด
!pip install -q -U \
    transformers \
    accelerate \
    peft \
    bitsandbytes \
    datasets \
    trl \
    huggingface_hub \
    wandb \
    requests \
    psutil \
    GPUtil

print("✅ ติดตั้ง dependencies เสร็จสมบูรณ์")
print("")
print("⚠️ สำคัญมาก: กรุณา Restart Runtime")
print("   Runtime > Restart runtime")
print("")
print("หลังจาก restart แล้ว ให้รัน Cell 2 เป็นต้นไป (ข้าม Cell 1)")

Cell 2: Configuration และ Imports

In [ ]:
import os
import json
import time
import requests
from datetime import datetime
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import psutil
try:
    import GPUtil
    GPU_AVAILABLE = True
except:
    GPU_AVAILABLE = False

print("✅ Import libraries สำเร็จ")

🔧 CONFIGURATION - กรอกข้อมูลของคุณที่นี่

In [ ]:
# Hugging Face Token
HF_TOKEN = ""  # กรอก HF Token ของคุณที่นี่

# Telegram Bot Configuration
TELEGRAM_BOT_TOKEN = ""  # กรอก Telegram Bot Token ที่นี่
TELEGRAM_CHAT_ID = ""  # กรอก Telegram Chat ID ที่นี่

# Weights & Biases (Optional - ถ้าไม่ใช้ให้เว้นว่าง)
WANDB_API_KEY = ""  # กรอก W&B API Key ถ้าต้องการใช้

# Model Configuration
BASE_MODEL = "dlnkgpt/dLNk-gpt-j-6b-agent-v1"
NEW_MODEL_NAME = "dlnkgpt/dLNk-gpt-j-6b-agent-v2-phase2"

# Training Hyperparameters (ปรับปรุงแล้ว!)
MAX_SEQ_LENGTH = 1024  # ลดจาก 2048 เพื่อความเร็ว
PER_DEVICE_TRAIN_BATCH_SIZE = 4  # เพิ่มจาก 1 → 4
GRADIENT_ACCUMULATION_STEPS = 8  # เพิ่มจาก 4 → 8
# Effective batch size = 4 × 8 = 32

LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 2  # ลดจาก 3 → 2 เพื่อเสร็จภายใน 24 ชม.
WARMUP_RATIO = 0.05
WEIGHT_DECAY = 0.01

# LoRA Configuration
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

# Logging & Checkpointing
LOGGING_STEPS = 10
SAVE_STEPS = 200  # ลดจาก 500 เพื่อ checkpoint บ่อยขึ้น
EVAL_STEPS = 200
TELEGRAM_NOTIFICATION_STEPS = 100  # ลดจาก 50 เพื่อลด spam

# W&B Configuration
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    WANDB_PROJECT = "dLNk-gpt-v2"
    WANDB_RUN_NAME = f"phase2-optimized-{datetime.now().strftime('%Y%m%d-%H%M')}"
else:
    WANDB_PROJECT = None
    WANDB_RUN_NAME = None

print("✅ Configuration เสร็จสมบูรณ์")
print(f"📦 Base Model: {BASE_MODEL}")
print(f"🎯 New Model: {NEW_MODEL_NAME}")
print(f"📏 Max Sequence Length: {MAX_SEQ_LENGTH}")
print(f"📊 Effective Batch Size: {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"🔄 Epochs: {NUM_TRAIN_EPOCHS}")
print(f"⏱️ ประมาณการเวลา: ~7-8 ชั่วโมง")

Cell 3: Telegram Notification Function

In [ ]:
def send_telegram(message):
    """ส่งข้อความผ่าน Telegram"""
    if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_ID:
        return
    
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {
            "chat_id": TELEGRAM_CHAT_ID,
            "text": message,
            "parse_mode": "HTML"
        }
        response = requests.post(url, data=data, timeout=10)
        if response.status_code == 200:
            print("✅ Telegram notification sent")
        else:
            print(f"⚠️ Telegram notification failed: {response.status_code}")
    except Exception as e:
        print(f"⚠️ Telegram error: {e}")

# ทดสอบ Telegram
send_telegram(f"🚀 <b>เริ่มการเทรน Phase 2 (v2.0 Optimized)</b>\\n\\n⏰ เวลาเริ่มต้น: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n📊 Effective Batch Size: {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}\\n⏱️ ประมาณการเวลา: ~7-8 ชั่วโมง")

print("✅ Telegram notification function ready")

Cell 4: GPU Check และ System Info

In [ ]:
print("🖥️ ตรวจสอบระบบ...")
print("=" * 60)

# CPU Info
print(f"💻 CPU: {psutil.cpu_count()} cores")
print(f"🧠 RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

# GPU Info
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM Total: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    print(f"💾 VRAM Available: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / (1024**3):.1f} GB")
    
    if GPU_AVAILABLE:
        gpus = GPUtil.getGPUs()
        if gpus:
            gpu = gpus[0]
            print(f"🔥 GPU Load: {gpu.load * 100:.1f}%")
            print(f"⚡ GPU Memory Used: {gpu.memoryUsed} MB / {gpu.memoryTotal} MB ({gpu.memoryUtil * 100:.1f}%)")
else:
    print("❌ GPU ไม่พร้อมใช้งาน!")
    raise RuntimeError("ต้องใช้ GPU ในการเทรน!")

print("=" * 60)
print("✅ ระบบพร้อมใช้งาน")

Cell 5: โหลด Base Model

In [ ]:
print("📥 กำลังโหลด base model...")
print(f"Model: {BASE_MODEL}")

# BitsAndBytesConfig สำหรับ 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# โหลด tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    token=HF_TOKEN,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# โหลด base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    token=HF_TOKEN,
    trust_remote_code=True
)

# เตรียมโมเดลสำหรับ k-bit training
base_model = prepare_model_for_kbit_training(base_model)

print("✅ โหลด base model สำเร็จ")
print(f"📊 Model parameters: {base_model.num_parameters() / 1e9:.2f}B")

Cell 6: ตั้งค่า LoRA

In [ ]:
print("🔧 กำลังตั้งค่า LoRA...")

# LoRA Configuration
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

# เพิ่ม LoRA adapters
model = get_peft_model(base_model, lora_config)

# แสดงจำนวน parameters ที่เทรน
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print("✅ ตั้งค่า LoRA สำเร็จ")
print(f"🎯 Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"📊 Total parameters: {total_params:,}")

Cell 7: โหลดและเตรียม Training Data

In [ ]:
print("📥 กำลังโหลด training data...")

# โหลด datasets
print("  - CodeAlpaca-20k...")
dataset1 = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

print("  - Python Code Instructions 18k...")
dataset2 = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")

print("  - Code Instructions 120k (subset)...")
dataset3 = load_dataset("iamtarun/code_instructions_120k_alpaca", split="train[:20000]")

# รวม datasets
combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3])

print(f"✅ โหลด training data สำเร็จ: {len(combined_dataset):,} examples")

# แบ่ง train/eval
split_dataset = combined_dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"📊 Train: {len(train_dataset):,} examples")
print(f"📊 Eval: {len(eval_dataset):,} examples")

# Formatting function
def format_instruction(example):
    """แปลง dataset เป็นรูปแบบที่ต้องการ"""
    if "instruction" in example and "output" in example:
        instruction = example.get("instruction", "")
        input_text = example.get("input", "")
        output = example.get("output", "")
        
        if input_text:
            text = f"### Instruction:\\n{instruction}\\n\\n### Input:\\n{input_text}\\n\\n### Response:\\n{output}"
        else:
            text = f"### Instruction:\\n{instruction}\\n\\n### Response:\\n{output}"
        
        return {"text": text}
    return {"text": ""}

# Format datasets
train_dataset = train_dataset.map(format_instruction, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(format_instruction, remove_columns=eval_dataset.column_names)

# แสดง sample
print("\\n📝 Sample training example:")
print("=" * 60)
print(train_dataset[0]["text"][:500] + "...")
print("=" * 60)

print("✅ เตรียม training data เสร็จสมบูรณ์")

Cell 8: ตั้งค่า Training Arguments

In [ ]:
print("🔧 กำลังตั้งค่า training arguments...")

training_args = TrainingArguments(
    output_dir="./dLNk-gpt-v2-phase2-optimized",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    logging_steps=LOGGING_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    report_to="wandb" if WANDB_API_KEY else "none",
    run_name=WANDB_RUN_NAME if WANDB_API_KEY else None,
    logging_dir="./logs",
    logging_first_step=True,
    disable_tqdm=False,
)

print("✅ Training arguments เสร็จสมบูรณ์")
print(f"📊 Total training steps: ~{len(train_dataset) // (PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS}")
print(f"⏱️ ประมาณการเวลา: ~7-8 ชั่วโมง")

Cell 9: สร้าง Trainer พร้อม Callbacks

In [ ]:
print("🔧 กำลังสร้าง trainer...")

# Custom Callback สำหรับ Telegram Notifications
from transformers import TrainerCallback

class TelegramCallback(TrainerCallback):
    def __init__(self, notification_steps=100):
        self.notification_steps = notification_steps
        self.start_time = time.time()
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.global_step % self.notification_steps == 0 and state.global_step > 0:
            elapsed_time = time.time() - self.start_time
            hours = int(elapsed_time // 3600)
            minutes = int((elapsed_time % 3600) // 60)
            
            loss = logs.get("loss", 0)
            lr = logs.get("learning_rate", 0)
            
            message = f"📊 <b>Step {state.global_step}</b>\\n\\n"
            message += f"📉 Loss: {loss:.4f}\\n"
            message += f"📈 LR: {lr:.2e}\\n"
            message += f"⏱️ เวลาที่ผ่านไป: {hours}h {minutes}m\\n"
            message += f"🔄 Epoch: {state.epoch:.2f}/{NUM_TRAIN_EPOCHS}"
            
            send_telegram(message)

# สร้าง Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    max_seq_length=MAX_SEQ_LENGTH,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
    dataset_text_field="text",
    callbacks=[
        TelegramCallback(notification_steps=TELEGRAM_NOTIFICATION_STEPS),
        EarlyStoppingCallback(early_stopping_patience=3)
    ]
)

print("✅ Trainer พร้อมใช้งาน")
print("✅ Early Stopping: Enabled (patience=3)")
print("✅ Telegram Notifications: Enabled")

Cell 10: เริ่มการเทรน

In [ ]:
print("=" * 60)
print("🚀 เริ่มการเทรน Phase 2 (v2.0 Optimized)...")
print(f"⏰ เวลาเริ่มต้น: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

send_telegram(f"🚀 <b>เริ่มการเทรน!</b>\\n\\n⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# เริ่มเทรน
trainer.train()

print("=" * 60)
print("✅ การเทรนเสร็จสมบูรณ์!")
print(f"⏰ เวลาสิ้นสุด: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

send_telegram(f"✅ <b>การเทรนเสร็จสมบูรณ์!</b>\\n\\n⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell 11: Save และ Upload โมเดล

In [ ]:
print("💾 กำลังบันทึกและอัปโหลดโมเดล...")

send_telegram("💾 เริ่มบันทึกและอัปโหลดโมเดล...")

try:
    # Save locally
    trainer.model.save_pretrained("./final_model")
    tokenizer.save_pretrained("./final_model")
    print("✅ บันทึกโมเดลในเครื่องสำเร็จ")
    
    # Upload to Hugging Face
    print(f"📤 กำลังอัปโหลดไปยัง {NEW_MODEL_NAME}...")
    trainer.model.push_to_hub(NEW_MODEL_NAME, token=HF_TOKEN)
    tokenizer.push_to_hub(NEW_MODEL_NAME, token=HF_TOKEN)
    
    print("✅ อัปโหลดโมเดลสำเร็จ!")
    print(f"🔗 URL: https://huggingface.co/{NEW_MODEL_NAME}")
    
    send_telegram(f"✅ <b>อัปโหลดโมเดลสำเร็จ!</b>\\n\\n🔗 <a href='https://huggingface.co/{NEW_MODEL_NAME}'>View Model</a>")
    
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")
    send_telegram(f"❌ <b>เกิดข้อผิดพลาดในการอัปโหลด:</b>\\n\\n{str(e)}")

Cell 12: ทดสอบโมเดล

In [ ]:
print("🧪 ทดสอบโมเดล...")

# Test prompts
test_prompts = [
    "Write a Python function to implement binary search:",
    "Explain SQL injection vulnerability and how to prevent it:",
    "Create a simple web scraper using BeautifulSoup:"
]

print("=" * 60)
for i, prompt in enumerate(test_prompts, 1):
    print(f"\\n🧪 Test {i}: {prompt}")
    print("-" * 60)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)
    print("-" * 60)

print("=" * 60)
print("✅ การทดสอบเสร็จสมบูรณ์!")

send_telegram("✅ <b>ทุกอย่างเสร็จสมบูรณ์!</b>\\n\\nโมเดลพร้อมใช้งาน 🎉")